# To be or what to be, that is the question
This notebook gives a solution for the following HackerRank challenge: https://www.hackerrank.com/challenges/to-be-what.

The goal is to predict the correct form of the "be" verb in a blanked text.

In [2]:
import re
from nltk.tokenize import wordpunct_tokenize
import numpy as np
from pprint import pprint
from time import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier, ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, make_scorer
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion

raw_corpus = open('corpus.txt', 'r', encoding='utf-8-sig').read()

## Training corpus pre-processing
A single book text file is provided for the traning corpus. Pre-processing is very light here.

In [3]:
title_match_regex = '\n{3,}\s+THE SECRET CACHE\n{3,}.*' # used to remove headers, toc, etc.
corpus = re.search(title_match_regex, raw_corpus, flags=re.M+re.S).group()
corpus = corpus.replace('\n', ' ') 
corpus = re.sub(r' {2,}', ' ', corpus) # replace multiple blanks by one
corpus = corpus.replace('----', '') # remove consecutive hyphens that we'll as a tag for the be verb
print('Corpus length after preprocessing:')
print('- {} characters\n- {} words'.format(len(corpus), len(corpus.split())))

Corpus length after preprocessing:
- 3301707 characters
- 585838 words


In [4]:
print('Corpus first 100 first words: \n {}'.format(' '.join(corpus.split()[:100])))

Corpus first 100 first words: 
 THE SECRET CACHE I THE BIRCH BARK LETTER On the river bank a boy sat watching the slender birch canoes bobbing about in the swift current. The fresh wind reddened his cheeks and the roaring of the rapids filled his ears. Eagerly his eyes followed the movements of the canoes daringly poised in the stream just below the tossing, foaming, white water. It was the first day of the spring fishing, and more exciting sport than this Indian white-fishing Hugh Beaupré had never seen. Three canoes were engaged in the fascinating game, two Indians in each. One knelt in the


## Training set creation
For the training set creation 

* targets are extracted ("be" forms)
* targets are removed from text
* a small text window (typically 10 words) is extracted around each target. This small text will be used for features creation

In [19]:
be_forms = ['am','are','were','was','is','been','being','be']
substitute = '----'
tokens = wordpunct_tokenize(corpus)

def find_targets(tokens):
    """ Return a list of found 'be' formed in a tokenized text """
    return [t for t in tokens if t in be_forms]
    
def remove_targets(tokens):
    """ Replace targets with a substitute in a tokenized text"""
    return [substitute if t in be_forms else t for t in tokens]

targets = find_targets(tokens)

tokens = remove_targets(tokens)

def create_windows(tokens, window_size=5):
    """ Create windows surrounding be forms. """
    for i in range(window_size):
        tokens.insert(0, 'BEGINNING')
    tokens.extend(['END']*window_size)
    contexts = []
    for i, word in enumerate(tokens):
        if word == substitute:
            window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
            window = ' '.join(window)
            contexts.append(window)    
    return np.array(contexts).reshape(-1, 1)

contexts = create_windows(tokens, window_size=2)
print('Number of "be" verb occurences: {}'.format(len(targets)))
print('Targets distribution:')
counts = np.unique(targets, return_counts=True)
print(list(zip(counts[0], counts[1])))
    
# Replace target names with integer label
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(targets)

Number of "be" verb occurences: 18647
Targets distribution:
[('am', 255), ('are', 1056), ('be', 2483), ('been', 1404), ('being', 327), ('is', 2867), ('was', 8012), ('were', 2243)]


## Modelling
Each text window is vectorized with a TF-IDF and modelled with a simple L2 regularized linear regression. Left and right parts of the window are dealt with separately before being concatenated. 

Note that the targets are strongly imbalanced.

In [20]:
contexts_train, contexts_test, y_train, y_test = train_test_split(contexts, y, test_size=0.3)

def avg_accuracy(y_test, y_pred):
    """ Average classes' prediction accuracy. It is a custom score that is not 
    remniscient of the training set class imbalance. """
    conf_mat = confusion_matrix(y_test, y_pred)
    return np.mean(conf_mat.diagonal()/conf_mat.sum(axis=1))

### Fit one model

In [21]:
from sklearn.preprocessing import FunctionTransformer

# Define function to extract left-part and right-part of the window separately
def texts_left(texts):
    cut_size = len(texts[0][0].split())//2
    return [' '.join(t[0].split()[:cut_size]) for t in texts]

def texts_right(texts):
    cut_size = len(texts[0][0].split())//2
    return [' '.join(t[0].split()[cut_size:]) for t in texts]

In [25]:
left_vectorizer = TfidfVectorizer(max_df=0.5, max_features=3000, min_df=2, stop_words=None)
right_vectorizer = TfidfVectorizer(max_df=0.5, max_features=3000, min_df=2, stop_words=None)
classifier = LogisticRegression(C=1., class_weight='balanced')

pipeline = Pipeline([
    ('union', FeatureUnion(transformer_list=[
                 ('left_bow', Pipeline([('cut_left', FunctionTransformer(texts_left)),
                                        ('vectorizer_left', left_vectorizer)
                                       ]),),
                 ('right_bow', Pipeline([('cut_right', FunctionTransformer(texts_right)),
                                         ('vectorizer_right', right_vectorizer)]))])),
    ('clf', classifier)])

pipeline.fit(contexts_train, y_train)

y_pred_train = pipeline.predict(contexts_train)
y_pred = pipeline.predict(contexts_test)

print('Average accuracy (train): {}'.format(avg_accuracy(y_train, y_pred_train)))
print('Average accuracy (test): {}'.format(avg_accuracy(y_test, y_pred)))

Average accuracy (train): 0.8035946975686428
Average accuracy (test): 0.6131110687135597


In [23]:
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))

             precision    recall  f1-score   support

         am       0.16      0.51      0.24        72
        are       0.52      0.49      0.50       339
         be       0.98      0.92      0.95       780
       been       0.98      0.94      0.96       449
      being       0.17      0.43      0.24        91
         is       0.51      0.49      0.50       828
        was       0.71      0.67      0.69      2391
       were       0.49      0.46      0.47       645

avg / total       0.69      0.66      0.67      5595



### Grid search

Grid search is used to find the best hyper-parameters for modelling. Should not be used before submission.

In [127]:
vectorizer = CountVectorizer()
classifier = LogisticRegression(class_weight='balanced')
pipeline = Pipeline([('vect', vectorizer), ('clf', classifier)])

parameters = {
    'vect__max_features': (5000, 10000),
    'vect__ngram_range': ((1,1), (1,2)),
    'clf__C': (0.5, 1.),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring=make_scorer(avg_accuracy))

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(contexts_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

y_pred_train = grid_search.predict(contexts_train)
y_pred = grid_search.predict(contexts_test)

print('Train score: {}'.format(avg_accuracy(y_train, y_pred_train)))
print('Test score: {}'.format(avg_accuracy(y_test, y_pred)))
print(classification_report(y_test, y_pred, target_names=target_encoder.classes_))

## Prediction on sample

In [29]:
# Read from file
sample = open('sample_input.txt', 'r', encoding='utf-8-sig').read().splitlines()
N = int(sample[0])
sample = sample[1]

In [27]:
# Read from STDIN (for submission on HackerRank)
import fileinput

f = fileinput.input()
N = int(f.readline())
sample = f.readline()

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [30]:
contexts_sample = create_windows(wordpunct_tokenize(sample))
sample_pred = target_encoder.inverse_transform(pipeline.predict(contexts_sample))
print('\n'.join(sample_pred))

were
was
are
were
is
were
